# PyProj

In [6]:
import pandas as pd
from urllib.parse import urlparse
import sys
import os
import cx_Oracle
from pprint import pprint

получаем информацию о компаниях, перечисленных в excel

In [72]:
def get_companies():
    tables = pd.read_html("https://www.nalog.ru/opendata/7707329152-reestropereldoc/", encoding="UTF-8", header=0,
                          index_col="№")
    frame = tables[0]
    registry = frame['Значение характеристики'].loc[8]  # url реестра

    url_validator(registry)

    company_list = pd.read_excel('C:\\code\\test.xlsx')['Компания'].values.tolist()
    print("содержимое excel файла")
    print(company_list)
    orders = pd.read_csv(registry, sep=';', header=0, index_col="GA")
    """GA	Порядковый номер
       GB	Наименование организации
       G1	Адрес компании
       G2	ИНН организации
       G3	Номер договора
       G4	Интернет сайт
       G5	Наличие автоустановщика для настроки програмного обеспечения при работе с электронной подписью
       """
    orders = orders[['GB', 'G1', 'G2', 'G3', 'G4']]
    comp_names = orders["GB"].str.lower()
    matching = comp_names.isin([x.lower() for x in company_list])
    for cn in company_list:
        if cn.lower() not in list(comp_names):
                print("No info for {}".format(cn.lower()))
    # print(orders[matching])

In [73]:
def url_validator(url):
    result = urlparse(url)
    nl = r"data.nalog.ru"
    f_format = ".csv"
    if not all([result.netloc == nl, result.path.endswith(f_format)]):
        print("некорректная ссылка на файл " + url)
        sys.exit(0)
    else:
        return

In [74]:
get_companies()

содержимое excel файла
['ЗАО ПФ СКБ Контур', 'ооо рпц партнер', 'ООО НПП Ижинформпроект', 'ЗАО Калуга Астрал', 'ООО РПЦ Партнер', 'ООО хаха', 'ОАО СКБ-банк']
No info for ооо хаха


Создаём связь с базой данных

In [75]:
cx_Oracle.init_oracle_client(lib_dir="C:\\code\\instantclient_19_9forever")

In [76]:
connectionString = '(description =(retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=lpdmasisaukuwfd_db202009tomch_high.adwc.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adwc.eucom-central-1.oraclecloud.com,OU=Oracle BMCS FRANKFURT,O=Oracle Corporation,L=Redwood City,ST=California,C=US")))'
conn = cx_Oracle.connect(r'ADMIN', r'M@i806M@i806', dsn=connectionString)

In [77]:
cursor = conn.cursor()